<a href="https://colab.research.google.com/github/tifftknguyen/Final-Project-456/blob/tiff/final-project-456.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiffany Nguyen
# Giselle Hernandez
# CECS 456 Sec 01 - Machine Learning
# Professor Wenlu Zhang
# Final Project

In [7]:
import os
import json

# Your Kaggle credentials
kaggle_token = {
    "username": "asdf",
    "key": "asdf"
}

# Create .kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Write kaggle.json
with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_token, f)

# Set correct permissions
os.chmod("/root/.kaggle/kaggle.json", 0o600)


In [8]:
!kaggle datasets download -d prasunroy/natural-images
!unzip -q natural-images.zip

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
 92% 315M/342M [00:06<00:00, 36.2MB/s]
100% 342M/342M [00:06<00:00, 52.5MB/s]


In [9]:
!kaggle datasets list | head

ref                                                          title                                               size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
wardabilal/spotify-global-music-dataset-20092025             Spotify Global Music Dataset (2009–2025)         1289021  2025-11-11 09:43:05.933000          14702        332  1.0              
rohiteng/amazon-sales-dataset                                Amazon Sales Dataset                             4037578  2025-11-23 14:29:37.973000           5568         76  1.0              
rohiteng/spotify-music-analytics-dataset-20152025            Spotify Music Analytics Dataset (2015–2025)      5080400  2025-12-04 05:17:45.537000           1684         42  1.0              
kundanbedmutha/exam-score-prediction-dataset 